# Python code for final project PHZ5305: Interaction matrix elements and Hartree-Fock

## Introduction.

This code is written for the final project of PHZ5305 (Nuclear Physic 1), which will provide the tool for reads the list of single-particle states and the me2b generated, and solves the Hartree-Fock Equation. 


## Import necessary packages.

`numpy` is imported for computational works. `pandas` is imported for reading the values of me2b from outer sources, and `itertools` for iterational job for lists, for example permutations. 

In [1]:
import numpy as np
import pandas as pd
import itertools as itr

## Define Class for Single-Particle States

In this block, the class for single-particles states are generated.

In the `__init__` block, the basic definition of `SP_States` are implied.

In the `nucleon_loop` block, the loop that generates `SP_list` is encoded.

In the `generate_lists` block, by using `nucleon_loop` function twice to make the list of single particle states. 

In [2]:
class SP_States:
    def __init__(self,A,Z):
        self.A = A                                     # Atomic Mass
        self.Z = Z                                     # Number of Protons
        self.N = A-Z                                   # Number of Neutrons
        self.i_degen = [2, 4, 2]                       # degeneracy for 0s1/2, 0p3/2, 0p1/2 shells
        self.i_numsh = [0, 1, 1]                       # Will be implied as l
    
    def nucleon_loop(self, SP_list, tz):
        i_shell = 0
        N_nucl  = 0                                    # Number of Nucleon
        I_nucl  = 0 
        
        if   tz == -0.5 : N_nucl = self.Z
        elif tz ==  0.5 : N_nucl = self.N
        
        index = len(SP_list)
        
        for itr in range(len(self.i_degen)):
            n     = 0                                  # shell's n
            l     = self.i_numsh[itr]                  # for first shell, l = 0, and for second and third shell, l = 1
            j     = (self.i_degen[itr]-1)/2            # from num. deg. states = 2J+1 -> J = (num. deg. states-1)/2 
            mj    = -j                                 # Starting from mj = -j
            if I_nucl == N_nucl: break                 # Break the loop when it reaches N_nucl
            while mj < j+1 :                           # Start iteration from mj = -j to mj = +j
                SP_list.append([index,n,l,j,mj,tz])    # Generate the single-particle states for protons
                mj     = mj     + 1                    # mj up
                I_nucl = I_nucl + 1
                index  = index  + 1                    # index   

        else: i_shell = i_shell+1
    
    def generate_lists_SP(self):
        SP_list = []                                   # Single Particle State's List
        
        self.nucleon_loop(SP_list, -0.5)               # Loop for Proton
        self.nucleon_loop(SP_list,  0.5)               # Loop for Neutron
        
        self.SP_list = SP_list
        return SP_list
        

## Define class for generate two-particle states from one-particle states, and antisymmetrize it.

In this block, the class that have an input of the list of single-particle states and generate the list of double-particle states.

In [3]:
class TB_States:
    def __init__(self,SP_States):
        self.A = SP_States.A                             # Atomic mass
        self.Z = SP_States.Z                             # Number of Protons
        self.N = SP_States.N                             # Number of Neutrons
        self.list_SP = SP_States.generate_lists_SP()     # Generate the Single Particle States
        
    def generate_lists_TB(self):                         # Generates the list of two-body states, not antisymmetrized yet.
        Ind_SP = list(np.arange(0,len(self.list_SP),1))  # Generate index for one-body states
        Ind_TB = [[i_SP, j_SP] for i_SP in Ind_SP for j_SP in Ind_SP if i_SP<j_SP and self.TB_analyzer([i_SP,j_SP])] 
                                                         # Generate the two-body index states, ignoring i=j cases
        self.Ind_TB = Ind_TB
        return Ind_TB
        
    def assymetrizer(self):                              # Assymetrize the results of generate_lists_TB.
        self.generate_lists_TB()                         # Get Two-Body states (not asymmetrized)
        Asym_Ind_TB = [list(itr.permutations(i_Ind_TB)) for i_Ind_TB in self.Ind_TB if self.TB_analyzer(i_Ind_TB)]
                                                         # Generate all the permutations in Ind_TB
        self.Asym_Ind_TB = Asym_Ind_TB
        return Asym_Ind_TB                               # Return permutated Two-body states
        
    def TB_analyzer(self, TB_State):                     # Input two-body state (a,b) and analyize the state is in Jp=0+
        Stat1 = self.list_SP[int(TB_State[0])]
        Stat2 = self.list_SP[int(TB_State[1])]
        
        n1, l1, j1, mj1 = Stat1[1], Stat1[2], Stat1[3], Stat1[4]
        n2, l2, j2, mj2 = Stat2[1], Stat2[2], Stat2[3], Stat2[4]
        
        
        
        if l1 == l2 and mj1 == -1*mj2 and j1 == j2 and n1==n2 : return True
        else: return False
        
        
        

## Define class for Hamiltonian 

This class has `onebody` function and `twobody` function

In [4]:
class Hamiltonian:
    def __init__(self, SP_States):
        self.A = SP_States.A                                # Atomic mass
        self.Z = SP_States.Z                                # Number of Protons
        self.N = SP_States.N                                # Number of Neutrons
        self.list_SP = SP_States.generate_lists_SP()        # Generate the Single Particle States
        self.list_TB = TB_States(SP_States).assymetrizer()  # Generate the Two-Particle States
        
    def onebody(self, SP_State):                            # Generate diagonal one-body energy for each single-particle states
        e_gap = 41*(self.A)^-0.33                           # Generate energy gap: 41*A^{1/3} MeV
        n, l = SP_State[1], SP_State[2]                     # Get quantum numbers n and l from SP_State
        return e_gap*(2*n + l + 1.5)                        # Return one-body energy
    
    def twobody(self, TB_State1, TB_State2, str_data):      # Generate two-body interaction, input is (a,b) and output is (c,d)
        with open(str_data, "r") as infilie:
            for line in infile:
                number = line.split()
        

## Main 

In [5]:
if __name__ == '__main__':
    O16 = SP_States(16,8)
    TB_O16 = TB_States(O16)
    A = TB_O16.generate_lists_TB()
    print(A)
    print(O16.SP_list)

[[0, 1], [0, 9], [1, 8], [2, 5], [2, 13], [3, 4], [3, 12], [4, 11], [5, 10], [6, 7], [6, 15], [7, 14], [8, 9], [10, 13], [11, 12], [14, 15]]
[[0, 0, 0, 0.5, -0.5, -0.5], [1, 0, 0, 0.5, 0.5, -0.5], [2, 0, 1, 1.5, -1.5, -0.5], [3, 0, 1, 1.5, -0.5, -0.5], [4, 0, 1, 1.5, 0.5, -0.5], [5, 0, 1, 1.5, 1.5, -0.5], [6, 0, 1, 0.5, -0.5, -0.5], [7, 0, 1, 0.5, 0.5, -0.5], [8, 0, 0, 0.5, -0.5, 0.5], [9, 0, 0, 0.5, 0.5, 0.5], [10, 0, 1, 1.5, -1.5, 0.5], [11, 0, 1, 1.5, -0.5, 0.5], [12, 0, 1, 1.5, 0.5, 0.5], [13, 0, 1, 1.5, 1.5, 0.5], [14, 0, 1, 0.5, -0.5, 0.5], [15, 0, 1, 0.5, 0.5, 0.5]]
